In [ ]:
import tensorflow as tf
import keras

In [ ]:
import pysmiles
import numpy as np

## Load some data

In [ ]:
import pyrfume

In [ ]:
molecules = pyrfume.load_data('leffingwell/molecules.csv', remote=True)
behavior = pyrfume.load_data('leffingwell/behavior.csv', remote=True)

In [ ]:
test_graph = pysmiles.read_smiles(molecules["IsomericSMILES"].iloc[0], explicit_hydrogen=True)

In [ ]:
test_graph.edges

In [ ]:
test_graph.nodes(data='element')

In [ ]:
test_graph.get_edge_data(5, 6)

In [ ]:
symbol_to_int = {'H': 0, 'He': 1, 'Li': 2, 'Be': 3, 'B': 4, 'C': 5, 'N': 6, 'O': 7, 'F': 8, 'Ne': 9, 'Na': 10, 'Mg': 11, 'Al': 12, 'Si': 13, 'P': 14, 'S': 15, 'Cl': 16, 'Ar': 17, 'K': 18, 'Ca': 19, 'Sc': 20, 'Ti': 21, 'V': 22, 'Cr': 23, 'Mn': 24, 'Fe': 25, 'Co': 26, 'Ni': 27, 'Cu': 28, 'Zn': 29, 'Ga': 30, 'Ge': 31, 'As': 32, 'Se': 33, 'Br': 34, 'Kr': 35, 'Rb': 36, 'Sr': 37, 'Y': 38, 'Zr': 39, 'Nb': 40, 'Mo': 41, 'Tc': 42, 'Ru': 43, 'Rh': 44, 'Pd': 45, 'Ag': 46, 'Cd': 47, 'In': 48, 'Sn': 49, 'Sb': 50, 'Te': 51, 'I': 52, 'Xe': 53, 'Cs': 54, 'Ba': 55, 'La': 56, 'Ce': 57, 'Pr': 58, 'Nd': 59, 'Pm': 60, 'Sm': 61, 'Eu': 62, 'Gd': 63, 'Tb': 64, 'Dy': 65, 'Ho': 66, 'Er': 67, 'Tm': 68, 'Yb': 69, 'Lu': 70, 'Hf': 71, 'Ta': 72, 'W': 73, 'Re': 74, 'Os': 75, 'Ir': 76, 'Pt': 77, 'Au': 78, 'Hg': 79, 'Tl': 80, 'Pb': 81, 'Bi': 82, 'Po': 83, 'At': 84, 'Rn': 85, 'Fr': 86, 'Ra': 87, 'Ac': 88, 'Th': 89, 'Pa': 90, 'U': 91, 'Np': 92, 'Pu': 93, 'Am': 94, 'Cm': 95, 'Bk': 96, 'Cf': 97, 'Es': 98, 'Fm': 99, 'Md': 100, 'No': 101, 'Lr': 102, 'Rf': 103, 'Db': 104, 'Sg': 105, 'Bh': 106, 'Hs': 107, 'Mt': 108, 'Ds ': 109, 'Rg ': 110, 'Cn ': 111, 'Nh': 112, 'Fl': 113, 'Mc': 114, 'Lv': 115, 'Ts': 116, 'Og': 117}

In [ ]:
def convertFromNetworkX(graph, maxNodes, maxEdges, embeddingDim):
    nodeEmbeddings = np.zeros((maxNodes, embeddingDim))
    edgeEmbeddings = np.zeros((maxEdges, embeddingDim))
    universalEmbedding = np.zeros((1, embeddingDim))
    adjacencyMatrix = np.zeros((maxNodes, maxNodes))

    # Populate node embeddings.
    for nodeNum, symbol in graph.nodes(data="element"):
        symbolInt = symbol_to_int[symbol]
        nodeEmbeddings[nodeNum][symbolInt] = 1.0
        
    # Populate edge embeddings and adjacency matrix.
    i = 0
    for start, end in graph.edges:
        edgeOrder = graph.get_edge_data(start, end)["order"]
        
        adjacencyMatrix[start][end] = 1.0
        adjacencyMatrix[end][start] = 1.0
    
    return nodeEmbeddings, edgeEmbeddings, universalEmbedding, adjacencyMatrix

In [ ]:
n, e, u, am = convertFromNetworkX(test_graph, 50, 50, 20)

In [ ]:
maxNodes = 50

In [ ]:
class GraphUpdate(keras.layers.Layer):
    def __init__(self, units=32, input_dim=32):
        super().__init__()
        self.w = self.add_weight(
            shape=(input_dim, units), initializer="random_normal", trainable=True
        )
        self.b = self.add_weight(shape=(units,), initializer="zeros", trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b